## Test out the LLM Deployment

### Requirements and Imports


In [ ]:
!pip install openai

### OpenAI Client Request


In [ ]:
base_url = "http://llm-deployment-predictor.text-to-image-demo.svc.cluster.local:8080/v1"  #don't forget to add the /v1 for the OpenAI endpoint
api_key = "EMPTY"


In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key=api_key,
    base_url=base_url
)

# Get model name
model_list = client.models.list()
model_name = model_list.data[0].id
print(f"Using model: {model_name}")

completion = client.chat.completions.create(
    model=model_name,
    messages=[
        # User message
        {
            "role": "user",
            "content": "Hi. Who are you?"
        }
    ],
    stream=True
)

for chunk in completion:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end='', flush=True)
print()


In [ ]:
from openai import OpenAI
from datetime import datetime


def get_current_time():
    """Returns the current time in UTC as an ISO 8601 string."""
    return datetime.utcnow().isoformat() + "Z"

client = OpenAI(
    api_key=api_key,
    base_url=base_url
)

# Example tool definition for tool calling
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "Returns the current time in UTC.",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": []
            }
        }
    }
]

# Get model name
model_list = client.models.list()
if not model_name:
    model_name = model_list.data[0].id
model_ids = ", ".join([model.id for model in model_list.data])
print(f"Using model: {model_name} from choices: {model_ids}")

completion = client.chat.completions.create(
    model=model_name,
    messages=[
        # User message
        {
            "role": "user",
            "content": "What time is it?"
        }
    ],
    stream=True,
    tools=tools
)
for chunk in completion:
    if chunk.choices[0].delta.tool_calls is not None:
        print("Tool call:", chunk.choices[0].delta.tool_calls)
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end='', flush=True)